In [2]:
import os
import re

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

import nltk.translate.bleu_score as bleu
from collections import Counter
from nltk import ngrams

##학습을 위한 병렬 corpus 가져오기
source와 target을 각각 병렬적으로 매치한 데이터셋

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -uq "/content/drive/MyDrive/fra-eng.zip" -d "/content" 

#영어와 프랑스어를 매치한 데이터 셋
#영어가 source 프랑스어가 target

In [4]:
def to_ascii(s):
  # 프랑스어 악센트(accent) 삭제
  # 예시 : 'déjà diné' -> deja dine
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
  # 악센트 제거 함수 호출
  sent = to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백 추가해서 구두점을 구분
  # ex) "I am a student." => "I am a student ."
  sent = re.sub(r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",")  영어랑 . ? ! , 제외하고 모두 지움
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  # 다수 개의 공백을 하나의 공백으로 치환
  sent = re.sub(r"\s+", " ", sent)
  return sent

In [5]:
num_samples =40000 #원래는 19만개의 문장이 있는데 4만개정도만 사용

def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], [] 
  #input을 3개 만듦 encoder는 output이 없으므로 encoder target은 만들 필요 x
  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines): #line 하나 안에 tab을 기준으로 source와 target을 구분하고 있음
      # source 데이터와 target 데이터를 tab을 기준으로 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line = [w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_in = [w for w in ("<sos> " + tar_line).split()] #line을 받아와서 sos 토큰을 넣어준다
      tar_line_out = [w for w in (tar_line + " <eos>").split()] #line을 받아와서 eos 토큰을 넣어준다.

      encoder_input.append(src_line)
      decoder_input.append(tar_line_in)
      decoder_target.append(tar_line_out)

      if i == num_samples - 1:
        break
  lines.close()
  return encoder_input, decoder_input, decoder_target

sents_en_in , sents_fra_in, sents_fra_out = load_preprocessed_data()

In [6]:
print('인코더의 입력 :',sents_en_in[15])
print('디코더의 입력 :',sents_fra_in[15])
print('디코더의 레이블 :',sents_fra_out[15])

인코더의 입력 : ['run', '.']
디코더의 입력 : ['<sos>', 'prenez', 'vos', 'jambes', 'a', 'vos', 'cous', '!']
디코더의 레이블 : ['prenez', 'vos', 'jambes', 'a', 'vos', 'cous', '!', '<eos>']


##Seq2Seq에 넣기 위해서 tokenize

영어와 프랑스어 각각에 대해서 tokenize함


In [7]:
#tokenizer 안에 filters는 문장 내에서 ""안에 것을 filtering함 우리는 이미 전처리 다해놔서 거를 것 없음 lower도 이미 해놓음

tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)


encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")
#padding에는 2가지 pre, post 존재 post는 0들을 뒤에 채우는 것

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [8]:
encoder_input[0]

array([31,  1,  0,  0,  0,  0,  0,  0], dtype=int32)

In [9]:
#영어는 문장 최대 길이가 8인 반면 프랑스어는 최대 길이가 16

print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (40000, 8)
디코더의 입력의 크기(shape) : (40000, 16)
디코더의 레이블의 크기(shape) : (40000, 16)


In [10]:
#word_index는 각 단어에 대한 index를 매칭해서 dictionary로 반환
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 5240, 프랑스어 단어 집합의 크기 : 9048


In [11]:
#word_index는 단어 - 인덱스 순의 dictionary
#index_word는 그 반대 아래의 딕셔너리는 이후 예측값과 실제값 예측에 사용

src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [12]:
#33000개의 문장을 무작위로 shuffle
indices = np.arange(num_samples)
np.random.shuffle(indices)
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [13]:
val_num=int(0.1*num_samples) #10% 만큼 test에 사용 split 해준다
encoder_input_train = encoder_input[:-val_num]
decoder_input_train = decoder_input[:-val_num]
decoder_target_train = decoder_target[:-val_num]

encoder_input_test = encoder_input[-val_num:]
decoder_input_test = decoder_input[-val_num:]
decoder_target_test = decoder_target[-val_num:]

In [14]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (36000, 8)
훈련 target 데이터의 크기 : (36000, 16)
훈련 target 레이블의 크기 : (36000, 16)
테스트 source 데이터의 크기 : (4000, 8)
테스트 target 데이터의 크기 : (4000, 16)
테스트 target 레이블의 크기 : (4000, 16)


##본격적인 lstm Seq2Seq 모델링

functional API로 쌓음  아래에 functional API에 대한 설명 있음

https://www.tensorflow.org/guide/keras/functional?hl=ko

In [15]:
embedding_dim = 64
hidden_units = 64

In [16]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) 
enc_masking = Masking(mask_value=0.0)(enc_emb) 

# 첫 번째 lstm
encoder_lstm1 = LSTM(hidden_units, return_sequences=True,return_state=True)  #return_sequences가 추가되었는데 2번째 lstm으로 전달하기 위해서 sequeucne를 열어준다.
encoder_outputs1, state_h1, state_c1 = encoder_lstm1(enc_masking) # 첫번째 lstm cell을 돌림 그리고 각각 output, h,c state 저장
encoder_states1 = [state_h1, state_c1] # 인코더의 은닉 상태와 셀 상태를 저장


#두 번째 lstm
encoder_lstm2 = LSTM(hidden_units, return_state=True)
encoder_outputs2, state_h2, state_c2 = encoder_lstm2(encoder_outputs1) #output을 lstm2에 넣어준다
encoder_states2 = [state_h2,state_c2]


In [17]:
# 디코더
decoder_inputs = Input(shape=(None,))
#dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb_layer = Embedding(tar_vocab_size, embedding_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm1 = LSTM(hidden_units, return_sequences=True, return_state=True)  
decoder_lstm2 = LSTM(hidden_units, return_sequences=True, return_state=True)  
# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs1, _, _ = decoder_lstm1(dec_masking,
                                     initial_state=encoder_states1)

decoder_outputs2, _, _ = decoder_lstm2(decoder_outputs1,
                                     initial_state=encoder_states2)
# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax') #vocab size만큼의 단어 분포 나오고 거기서 softmax
decoder_outputs = decoder_dense(decoder_outputs2) #output을 dense에 넣어줌

# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
#sparse cross entropy는 결과 값을 one-hot coding하지 않았을 때 cross entropy를 구해주는 loss function

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 64)     335360      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 64)     579072      ['input_2[0][0]']                
                                                                                              

In [19]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
282/282 [==============================] - 45s 80ms/step - loss: 4.8270 - acc: 0.6026 - val_loss: 2.6235 - val_acc: 0.6071
Epoch 2/50
282/282 [==============================] - 14s 49ms/step - loss: 2.4007 - acc: 0.6072 - val_loss: 2.1897 - val_acc: 0.6071
Epoch 3/50
282/282 [==============================] - 14s 49ms/step - loss: 2.0269 - acc: 0.6367 - val_loss: 1.8988 - val_acc: 0.7107
Epoch 4/50
282/282 [==============================] - 14s 49ms/step - loss: 1.7930 - acc: 0.7331 - val_loss: 1.7239 - val_acc: 0.7391
Epoch 5/50
282/282 [==============================] - 14s 49ms/step - loss: 1.6547 - acc: 0.7415 - val_loss: 1.6247 - val_acc: 0.7435
Epoch 6/50
282/282 [==============================] - 14s 49ms/step - loss: 1.5683 - acc: 0.7483 - val_loss: 1.5574 - val_acc: 0.7508
Epoch 7/50
282/282 [==============================] - 14s 49ms/step - loss: 1.5047 - acc: 0.7545 - val_loss: 1.5045 - val_acc: 0.7561
Epoch 8/50
282/282 [==============================] - 14s 49ms

## 학습 이후 학습 된 모델로 machine translation 실행

In [111]:
# 인코더는 train에서 사용되고 이미 train된 애들을 그대로 가져옵니다
encoder_model = Model(encoder_inputs, [encoder_states1]+[encoder_states2])

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h1 = Input(shape=(hidden_units,))
decoder_state_input_c1 = Input(shape=(hidden_units,))
decoder_states_inputs1 = [decoder_state_input_h1, decoder_state_input_c1]

decoder_state_input_h2 = Input(shape=(hidden_units,))
decoder_state_input_c2 = Input(shape=(hidden_units,))
decoder_states_inputs2 = [decoder_state_input_h2, decoder_state_input_c2]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 첫 번째 lstm
dec_outputs1, state_h1, state_c1 = decoder_lstm1(dec_emb2, initial_state=decoder_states_inputs1)
dec_states1 = [state_h1, state_c1]

#두 번째 lstm
dec_outputs2, state_h2, state_c2 = decoder_lstm2(dec_outputs1, initial_state=decoder_states_inputs2)
dec_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
dec_outputs2 = decoder_dense(dec_outputs2) #output을 desne에 넣어서 확률 처리

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs1+decoder_states_inputs2,[dec_outputs1]+dec_states1 + [dec_outputs2] + dec_states2)

In [112]:
#디코더를 컨트롤하기 위한 함수, test과정에서 사용할 함수

def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>'] #시작토큰의 inex를 시작에 넣어줌

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    out,h1,c1,output_tokens,h2, c2 = decoder_model.predict([target_seq] + states_value) 

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
    if (sampled_char == '<eos>' or
        len(decoded_sentence) > 50):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h1, c1, h2, c2]

  return decoded_sentence

In [113]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_to_index['<sos>'] and encoded_word != tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence

In [114]:
for seq_index in [6,3, 50, 100, 300, 1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print('BLEU :',bleu.sentence_bleu(list(map(lambda ref: ref.split(), seq_to_tar(decoder_input_train[seq_index]))),decoded_sentence[1:-5].split()))
  print("-"*50)

입력문장 : he lost everything . 
정답문장 : il a tout perdu . 
번역문장 : il etait des excuses . 
BLEU : 0.668740304976422
--------------------------------------------------


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


입력문장 : tell us a joke . 
정답문장 : raconte nous une blague . 
번역문장 : veuillez vous soyez bon ! 
BLEU : 0
--------------------------------------------------
입력문장 : i d never hire tom . 
정답문장 : je n engagerais jamais tom . 
번역문장 : je ne peux pas a l interieur . 
BLEU : 0.7071067811865476
--------------------------------------------------
입력문장 : no one likes me . 
정답문장 : personne ne m apprecie . 
번역문장 : personne ne peut le francais . 
BLEU : 0.6389431042462724
--------------------------------------------------
입력문장 : you re weak . 
정답문장 : vous etes faible . 
번역문장 : tu es fort . 
BLEU : 0.7071067811865476
--------------------------------------------------
입력문장 : i m very lazy . 
정답문장 : je suis tres paresseux . 
번역문장 : je suis tres occupee . 
BLEU : 0.668740304976422
--------------------------------------------------


In [115]:
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",decoded_sentence[1:-5])
  print('BLEU :',bleu.sentence_bleu(list(map(lambda ref: ref.split(), seq_to_tar(decoder_input_test[seq_index]))),decoded_sentence[1:-5].split()))

  print("-"*50)

입력문장 : tom grinned . 
정답문장 : tom souriait . 
번역문장 : tom a des jours . 
BLEU : 0.7952707287670506
--------------------------------------------------


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


입력문장 : i rewrote it . 
정답문장 : je le reecrivis . 
번역문장 : je l ai fait . 
BLEU : 0.7952707287670506
--------------------------------------------------
입력문장 : turn right there . 
정답문장 : tournez la devant a droite . 
번역문장 : salut ca . 
BLEU : 0.7598356856515925
--------------------------------------------------
입력문장 : no one can stop me . 
정답문장 : personne ne peut m arreter . 
번역문장 : personne ne peut etre faire . 
BLEU : 0.6389431042462724
--------------------------------------------------
입력문장 : what was that thing ? 
정답문장 : qu etait ce ? 
번역문장 : qu est ce que il a la maison ? 
BLEU : 0.6865890479690392
--------------------------------------------------
